torch 불러오기

In [15]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.9.0
GPU 사용 가능 여부: True


In [16]:
data_dir = './archive/rps'

In [17]:
batch_size = 100
num_epochs = 10
learning_rate = 0.0001

In [18]:
class RPSDataset(Dataset):
    def __init__(self, data_dir, mode, transform=None):
        self.all_data = sorted(glob.glob(os.path.join(data_dir, mode, '*', '*')))
        self.transform = transform
    
    def __getitem__(self, index):
        # Step 1: 반환할 이미지 경로 정의 및 이미지 로드
        ## 코드 시작 ##
        data_path = self.all_data[index]    # 위의 설명 Step 1의 1. 을 참고하여 None을 채우세요.
        img = Image.open(data_path).convert('RGB')          # 위의 설명 Step 1의 2. 를 참고하여 None을 채우세요.
        if self.transform:
            img = self.transform(img)
            
            # 위의 설명 Step 1의 3. 을 참고하여 None을 채우세요.
        

        ## 코드 종료 ##
        
        # Step 2: 이미지에 대한 label 정의
        ## 코드 시작 ##
        if(os.path.basename(data_path).startswith('paper')):
            label = 0
        elif(os.path.basename(data_path).startswith('rock')):
            label = 1
        elif(os.path.basename(data_path).startswith('scissors')):
            label = 2
                # 위의 설명 Step 2 를 참고하여 None을 채우세요.
        ## 코드 종료 ##
        return img, label
    
    def __len__(self):
        ## 코드 시작 ##
        length = len(self.all_data)
        ## 코드 종료 ##
        return length
        

In [19]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(120, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([120, 120]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
train_data = RPSDataset(data_dir='./archive/rps', mode='train', transform=data_transforms['train'])
val_data = RPSDataset(data_dir='./archive/rps', mode='val', transform=data_transforms['val'])
test_data = RPSDataset(data_dir='./archive/rps', mode='test', transform=data_transforms['val'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=1, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False, drop_last=True)

In [20]:
len(train_data)

2520

In [21]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv 구현
        self.conv = nn.Sequential(
            ## 코드 시작 ##
            nn.Conv2d(3, 32, 3),    # conv_1 해당하는 층
            nn.BatchNorm2d(32),    # batch_norm_1 해당하는 층
            nn.ReLU(),    # ReLU_1 해당하는 층
            nn.MaxPool2d(2),    # maxpool_1 해당하는 층
            
            nn.Conv2d(32, 64, 3),    # conv_2 해당하는 층
            nn.BatchNorm2d(64),    # batch_norm_2 해당하는 층
            nn.ReLU(),    # ReLU_2 해당하는 층
            nn.MaxPool2d(2),    # maxpool_2 해당하는 층
            
            nn.Conv2d(64, 128, 3),    # conv_3 해당하는 층
            nn.BatchNorm2d(128),    # batch_norm_3 해당하는 층 
            nn.ReLU(),    # ReLU_3 해당하는 층
            nn.MaxPool2d(2),    # maxpool_3 해당하는 층
            
            nn.Conv2d(128, 128, 3),    # conv_4 해당하는 층
            nn.BatchNorm2d(128),    # batch_norm_4 해당하는 층
            nn.ReLU(),    # ReLU_4 해당하는 층
            nn.MaxPool2d(2),    # maxpool_4 해당하는 층
            ## 코드 종료 ##
        )
        
        # self.fc 구현
        ## 코드 시작 ##
        self.fc1 = nn.Linear(3200, 512)
        self.fc2 = nn.Linear(512, 3)
        ## 코드 종료 ##
    
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [22]:
def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    best_loss = 9999999
    for epoch in range(num_epochs):
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)  # 위의 설명 1. 을 참고하여 None을 채우세요.
            loss = criterion(outputs, labels)     # 위의 설명 2. 를 참고하여 None을 채우세요.

            optimizer.zero_grad()            # Clear gradients: 위의 설명 3. 을 참고하여 None을 채우세요.
            loss.backward()            # Gradients 계산: 위의 설명 3. 을 참고하여 None을 채우세요.
            optimizer.step()            # Parameters 업데이트: 위의 설명 3. 을 참고하여 None을 채우세요.
            ## 코드 종료 ##

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()

            if (i+1) % 3 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100))

        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)

In [23]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            ## 코드 종료 ##
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Validation #{}  Accuracy: {:.2f}%  Average Loss: {:.4f}'.format(epoch, correct / total * 100, avrg_loss))
    model.train()
    return avrg_loss

In [24]:
def test(model, data_loader, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            ## 코드 종료 ##
            _, argmax = torch.max(outputs, 1)    # max()를 통해 최종 출력이 가장 높은 class 선택
            total += imgs.size(0)
            correct += (labels == argmax).sum().item()

        print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total * 100))
    model.train()

In [25]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    ## 코드 시작 ##
    torch.save(check_point, output_path)
    ## 코드 종료 ##

In [26]:
torch.manual_seed(7777) # 일관된 weight initialization을 위한 random seed 설정
## 코드 시작 ##
model = SimpleCNN()          # 위의 설명 1. 을 참고하여 None을 채우세요.
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()      # 위의 설명 2. 를 참고하여 None을 채우세요.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)      # 위의 설명 3. 을 참고하여 None을 채우세요.
## 코드 종료 ##

model = model.to(device)
val_every = 1
saved_dir = './saved/SimpleCNN'

In [27]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

Start training..
Epoch [1/10], Step [3/25], Loss: 0.9740, Accuracy: 40.00%
Epoch [1/10], Step [6/25], Loss: 0.7315, Accuracy: 69.00%
Epoch [1/10], Step [9/25], Loss: 0.5976, Accuracy: 85.00%
Epoch [1/10], Step [12/25], Loss: 0.5266, Accuracy: 82.00%
Epoch [1/10], Step [15/25], Loss: 0.3078, Accuracy: 98.00%
Epoch [1/10], Step [18/25], Loss: 0.2883, Accuracy: 95.00%
Epoch [1/10], Step [21/25], Loss: 0.1685, Accuracy: 100.00%
Epoch [1/10], Step [24/25], Loss: 0.2056, Accuracy: 96.00%
Start validation #1
Validation #1  Accuracy: 69.70%  Average Loss: 0.8113
Best performance at epoch: 1
Save model in ./saved/SimpleCNN
Epoch [2/10], Step [3/25], Loss: 0.1013, Accuracy: 98.00%
Epoch [2/10], Step [6/25], Loss: 0.1262, Accuracy: 97.00%
Epoch [2/10], Step [9/25], Loss: 0.0667, Accuracy: 99.00%
Epoch [2/10], Step [12/25], Loss: 0.0637, Accuracy: 99.00%
Epoch [2/10], Step [15/25], Loss: 0.0608, Accuracy: 99.00%
Epoch [2/10], Step [18/25], Loss: 0.0359, Accuracy: 100.00%
Epoch [2/10], Step [21/25]

In [28]:
model_path = './saved/SimpleCNN/best_model.pt'
# model_path = './saved/pretrained/SimpleCNN/best_model.pt' # 모델 학습을 끝까지 진행하지 않은 경우에 사용
model = SimpleCNN().to(device)   # 아래의 모델 불러오기를 정확히 구현했는지 확인하기 위해 새로 모델을 선언하여 학습 이전 상태로 초기화

## 코드 시작 ##
checkpoint = torch.load(model_path, map_location=device)    # 위의 설명 1. 을 참고하여 None을 채우세요.
state_dict = model.load_state_dict(checkpoint['net'])    # 위의 설명 2. 를 참고하여 None을 채우세요.
model.eval()               # 위의 설명 3. 을 참고하여 None을 채우세요.
## 코드 종료 ##

SimpleCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): MaxPool2d(kernel_size=2, stride=2, paddi

In [29]:
test(model, test_loader, device)

Start test..
Test accuracy for 372 images: 83.06%


Resnet
모델


In [ ]:
new_model = torchvision.models.resnet50(pretrained=True)

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_data = RPSDataset(data_dir='./archive/rps', mode='train', transform=data_transforms['train'])
val_data = RPSDataset(data_dir='./archive/rps', mode='val', transform=data_transforms['val'])
test_data = RPSDataset(data_dir='./archive/rps', mode='test', transform=data_transforms['val'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
for param in new_model.parameters():
    param.requires_grad = False

num_ftrs = new_model.fc.in_features
## 코드 시작 ##
new_model.fc = nn.Linear(2048, 2)    # 위의 설명 1. 을 참고하여 None을 채우세요.
criterion = torch.nn.CrossEntropyLoss()       # 위의 설명 2. 를 참고하여 None을 채우세요.
new_model = new_model.to(device)
optimizer = torch.optim.Adam(new_model.parameters(), lr=learning_rate)       # 위의 설명 3. 을 참고하여 None을 채우세요.
## 코드 종료 ##
val_every = 1
saved_dir = './saved/ResNet50'

In [ ]:
train(num_epochs, new_model, train_loader, criterion, optimizer, saved_dir, val_every, device)